In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin


In [ ]:
### Standard Imports
import numpy as np
import re
import sys
from collections import Counter

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils

In [ ]:
### Text Parameters
start_token = '<cls>'
end_token = '<eos>'
pad_token = '<pad>'

### Model Parameters
window_len = 15
batch_size = 64
embedding_dim = 128
rnn_dim = 256
learn_rate = 0.001
epochs = 30

In [ ]:
### Load Data
corpus = utils.load_corpus()

In [ ]:
### Pre-Processing Text
words = utils.preprocess_text(corpus, fun_list = [utils.to_lower, utils.decontraction, utils.remove_punct], keep = '\<|\>')
words = re.sub('\n',' \n ', words)
words = re.split(' +', words) #Tokenising

word_count = Counter(words) # Assumes end_token is already in the corpus
word_count[start_token] = 0
word_count[pad_token] = 0

#Reference Dictionaries to convert one-hot index to string and vice versa
index_to_vocab = {i: k for i, k in enumerate(word_count.keys())}
vocab_to_index = {k: i for i, k in enumerate(word_count.keys())}

songs = ' '.join(words)
songs = songs.split(' \n \n <eos> \n \n ')
songs = [song.split(' ') for song in songs]
songs = [[pad_token]*(window_len-1) + [start_token] + song + [end_token] + [pad_token]*(window_len-1) for song in songs]
songs_token_ind = [[vocab_to_index.get(x) for x in song] for song in songs]

In [ ]:
### Creating Dataset
x_encoder = []
x_decoder = []
y = []
vocab_size = len(word_count)

for song in songs_token_ind:
  for i in range(len(song)-window_len):
    x_encoder.append(song[(i+1):(i+window_len+1)])
    x_decoder.append(song[i:(i+window_len)])
    y.append(song[i+window_len])

In [ ]:
# rand_int = np.random.randint(0, len(x_encoder), 1)[0]
# print([index_to_vocab.get(x) for x in x_encoder[rand_int]])
# print([index_to_vocab.get(x) for x in x_decoder[rand_int]])
# print(index_to_vocab.get(y[rand_int]))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(((x_encoder, x_decoder), y))
dataset = dataset.batch(batch_size)
dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Encoder
encoder_input = layers.Input(shape=(window_len), name = 'encoder_input')
encoder_embedded = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length = window_len, name = 'encoder_embedding')(
    encoder_input
)

# Return state in addition to output
encoder_output, encoder_state = layers.SimpleRNN(rnn_dim, return_state=True, name = "encoder_rnn")(
    encoder_embedded
)

# Decoder
decoder_input = layers.Input(shape=(window_len), name = 'decoder_input')
decoder_embedded = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length = window_len, name = 'decoder_embedding')(
    decoder_input
)

# Pass the encoder state to a new RNN, as initial state
decoder_output = layers.SimpleRNN(rnn_dim, name="decoder_rnn")(
    decoder_embedded, initial_state=[encoder_state]
)
output = layers.Dense(vocab_size, name = 'output', activation = 'softmax')(decoder_output)

model = tf.keras.Model((encoder_input, decoder_input), output, name = 'autoencoder')
model.summary()

Model: "autoencoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 15)]         0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, 15)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 15, 128)      134272      ['encoder_input[0][0]']          
                                                                                                  
 decoder_embedding (Embedding)  (None, 15, 128)      134272      ['decoder_input[0][0]']          
                                                                                        

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate))

In [ ]:
result = model.fit(x = dataset, epochs = epochs, shuffle = True)

Epoch 1/30
226/226 [==============================] - 15s 50ms/step - loss: 4.3260
Epoch 2/30
226/226 [==============================] - 7s 29ms/step - loss: 2.8950
Epoch 3/30
226/226 [==============================] - 7s 29ms/step - loss: 2.1413
Epoch 4/30
226/226 [==============================] - 7s 29ms/step - loss: 1.7846
Epoch 5/30
226/226 [==============================] - 7s 30ms/step - loss: 1.5383
Epoch 6/30
226/226 [==============================] - 7s 30ms/step - loss: 1.2804
Epoch 7/30
226/226 [==============================] - 7s 30ms/step - loss: 1.1500
Epoch 8/30
226/226 [==============================] - 7s 30ms/step - loss: 0.9224
Epoch 9/30
226/226 [==============================] - 7s 29ms/step - loss: 0.8420
Epoch 10/30
226/226 [==============================] - 7s 29ms/step - loss: 0.7114
Epoch 11/30
226/226 [==============================] - 7s 30ms/step - loss: 0.6677
Epoch 12/30
226/226 [==============================] - 7s 29ms/step - loss: 0.7429
Epoch 13/30


In [ ]:
def generate_text(model, start_string, num_generate = 1000, temperature=1.0, random_seed = 2022):
    # Converting our start string to numbers (vectorizing).
    input_indices = [vocab_to_index.get(s) for i, s in enumerate(start_string) if i < window_len-1]
    input_indices = [vocab_to_index.get(pad_token)]*(window_len - len(input_indices)-1) + [vocab_to_index.get(start_token)] + input_indices

    x = tf.expand_dims(input_indices, 0)

    # Empty string to store our results.
    text_generated = []

    # Here batch size == 1.
    model.reset_states()
    for word_index in range(num_generate):
        prediction = model.predict([x,x])

        # Using a categorical distribution to predict the character returned by the model.
        prediction = prediction / temperature
        predicted_id = tf.random.categorical(prediction, num_samples=1, seed = random_seed)[-1,0]
        
        # We pass the series of previous words (up to window length) as the next input to the model
        # along with the previous hidden state.
        input_index = tf.cast(tf.expand_dims([predicted_id], 0), tf.int32)
        x = tf.concat([x[:,1:],input_index], 1)
        
        pred_word = index_to_vocab[predicted_id.numpy()]
        text_generated.append(pred_word)
        if pred_word == end_token:
          break
    
    return (' '.join(start_string) + ' ' + ' '.join(text_generated)), text_generated

In [ ]:
result_str, result = generate_text(model, start_string=['<verse>','\n','step','by','step'], num_generate=30, temperature=1.0)

In [ ]:
print(result_str)

<verse> 
 step by step by look wind lift unfurled progress inspiration yea going towards said prove memory feel my twinkling hands cairo four homely shore not dawns seem generation summing cooking in real guess
